<a href="https://colab.research.google.com/github/joshdohler/FCTT_Project_3/blob/Diego/Project_3_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read CSV file containing training data Temp: 20ºC
df = pd.read_csv('Model_Training_Data_20.csv')
t = np.array(df['Time (s)'][:])
I = np.array(df['Current (A)'][:])
V = np.array(df['Voltage (V)'][:])

# Plot Data - Current
plt.figure(1, figsize=(10, 6))
plt.plot(t, I, label='Current (A)', color='blue')
plt.xlabel('Time (s)')
plt.ylabel('Current (A)')
plt.title('Discharge Pulses')
plt.grid(True)

# Plot Data - Voltage
plt.figure(2, figsize=(10, 6))
plt.plot(t, V, label='Voltage (V)', color='blue')
plt.xlabel('Time (s)')
plt.ylabel('Voltage (V)')
plt.title('Discharge Pulses')
plt.grid(True)

def identifypulses(I, threshold=0.5, pulse_duration=10):
    pulses = []
    in_pulse = False
    pulse_start = 0

    for i in range(1, len(I)):
        if abs(I[i] - I[i-1]) > threshold:
            if not in_pulse:
                pulse_start = i - 1
                in_pulse = True
        elif in_pulse:
            if i - pulse_start >= pulse_duration * 100:  # Check if pulse duration is approximately 10 seconds
                pulses.append((pulse_start, i))
                in_pulse = False
    return pulses

# Analyze pulses for the full data range
pulses = identifypulses(I)
pulsestimes = []

for start, end in pulses:
  tpulsestart = t[start]
  tpulsesend = t[end-1]
  pulsestimes.append((tpulsestart, tpulsesend))

# Divide list: 8 subsections - 8 SOC levels
nSOCs = 8
nPulsesSOC = 8
SOCpulseintervals = [[] for _ in range(nSOCs)]

for i, (tstart, tend) in enumerate(pulsestimes):
    SOCindex = i // nPulsesSOC
    SOCpulseintervals[SOCindex].append((tstart, tend))

# Print the identified pulse intervals for each SOC range
for idx, pulses in enumerate(SOCpulseintervals):
    SOCstart = 90 - idx * 10
    SOCend = 100 - idx * 10
    print(f"SOC Range: {SOCstart}% - {SOCend}%")
    for pulse_idx, (start, end) in enumerate(pulses):
        print(f"Pulse {pulse_idx + 1}: Start Time: {start}, End Time: {end}")

# SOCind: list to hold results
SOCind = []

pp1 = 4
pp2 = 13

for i in range(8):
    SOC = SOCpulseintervals[i]
    Pulse = []
    plt.figure(pp1)
    plt.figure(pp2)
    tpulse = np.zeros((1000, 8))
    Ipulse = np.zeros((1000, 8))
    Vpulse = np.zeros((1000, 8))
    for j in range(8):
        pulserange = SOC[j]
        # Use NumPy indexing to find matching time indices
        k_indices = np.where((t >= pulserange[0]) & (t <= pulserange[1]))[0]
        # Populate corresponding pulse data
        tpulse[:len(k_indices), j] = t[k_indices]
        Ipulse[:len(k_indices), j] = I[k_indices]
        Vpulse[:len(k_indices), j] = V[k_indices]

        # Plot
        plt.figure(pp1)
        plt.plot(tpulse[:, j], Ipulse[:, j], label=f'Pulse {j+1}')
        plt.title(f'SOC {i}')
        plt.xlabel('Time (s)')
        plt.ylabel('Current (A)')
        plt.legend()
        plt.figure(pp2)
        plt.plot(tpulse[:, j], Vpulse[:, j], label=f'Pulse {j+1}')
        plt.title(f'SOC {i}')
        plt.xlabel('Time (s)')
        plt.ylabel('Voltage (V)')
        plt.legend()

        Pulse.append([j, np.copy(tpulse), np.copy(Ipulse), np.copy(Vpulse)])
    pp1 = pp1+1
    pp2 = pp2+1
    SOCind.append(Pulse)

plt.figure(3)
plt.plot(tpulse[:, 0], Vpulse[:, 0], label=f'Pulse {j+1}', color='red')
plt.title(f'SOC {i}')
plt.xlabel('Time (s)')
plt.ylabel('Voltage (V)')
plt.legend()